In [ ]:
token='67d95da7-f763-4fa3-9a9c-4b4edc2b548b'
json_str='{"studyIds":["blca_mskcc_solit_2014"],"pageSize":999999,"pageIndex":1,"attributesRangeFilters":[],"attributesEqualFilters":[],"mutationFilter":{"hugoGeneSymbols":[],"exacStart":0,"exadEnd":1,"vabundStart":0,"vabundEnd":1,"variantSource":[],"variantType":[],"variantClass":[],"sequencer":[],"sequencerSource":[],"searchStr":""}}'
host='https://peta.bgi.com/api'

启动python环境

In [ ]:


tryCatch({
    library(reticulate)
    use_condaenv("r-reticulate")

 },error=function(e){
     print(e)
 } )

调用pypeta获取数据

In [ ]:

tryCatch({
    pypyta<- import("pypeta")
    peta=pypyta$Peta(token=token,host=host)
    peta$set_data_restriction_from_json_string(json_str)
    mut<-peta$fetch_mutation_data()
    mut<-mut[which(mut$NCBI_Build != 'NA'),]
    cli=peta$fetch_clinical_data()
    names(cli)[names(cli) =='sampleId'] <- 'Tumor_Sample_Barcode'

 },error=function(e){
     print(e)
 } )


以上为PETA系统IRKernel报告模板的必备固定代码，maf格式的突变信息和tsv格式的临床信息分别以dataframe格式存储于变量mut和cli中。下方为用户自定义代码，以执行各类分析和可视化功能。

In [ ]:
tryCatch({
    library(maftools)    
    laml = read.maf(maf = mut,clinicalData=cli,verbose = F)

 },error=function(e){
     print(e)
 } )


# 数据选择条件

In [ ]:
tryCatch({
    json<-import("json")
    json$loads(json_str)
    #pypyta$filter_description(json_str) #python函数中的print被导向到标准输出，不在当前位置显示

 },error=function(e){
     print(e)
 } )


# 基因组特征

## 临床属性分布
暂未实现

## 样本级别变异分布

In [ ]:

tryCatch({
     #Shows sample summry.
    getSampleSummary(laml)
 },error=function(e){
     print(e)
 } )

## 基因水平变异分布

In [ ]:

tryCatch({
     #Shows gene summary.
    getGeneSummary(laml)
 },error=function(e){
     print(e)
 } )

## 变异图谱

In [ ]:

tryCatch({
     suppressMessages(suppressWarnings(oncoplot(maf = laml, top = 10)))

 },error=function(e){
     print(e)
 } )



## 变异分布特性图示

In [ ]:
 tryCatch({
      suppressMessages(suppressWarnings(plotmafSummary(maf = laml, rmOutlier = TRUE, addStat = 'median', dashboard = TRUE, titvRaw = FALSE)))
 },error=function(e){
     print(e)
 } )

## Ti/Tv 

In [ ]:
tryCatch({
     laml.titv = titv(maf = laml, plot = FALSE, useSyn = TRUE)
     suppressMessages(suppressWarnings(plotTiTv(res = laml.titv)))
 },error=function(e){
     print(e)
 } )

## TMB 与 TCGA比较 

In [ ]:
tryCatch({
     laml.mutload =  suppressMessages(suppressWarnings(tcgaCompare(maf = laml, cohortName = 'Selected')))
 },error=function(e){
     print(e)
 } )

## 突变基因云图

In [ ]:
tryCatch({
      suppressMessages(suppressWarnings(geneCloud(input = laml, minMut = 2)))
 },error=function(e){
     print(e)
 } )

## 突变互斥共存特征分析

In [ ]:

tryCatch({    
     suppressMessages(suppressWarnings(somaticInteractions(maf = laml, top = 25, pvalue = c(0.05, 0.1))))

 },error=function(e){
     print(e)
 } )